In [1]:
import os

In [11]:
%pwd

'd:\\ML_Projects\\End-to-End ML Impleamentation\\End to end machine learning with mlflow'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML_Projects\\End-to-End ML Impleamentation\\End to end machine learning with mlflow'

In [5]:
os.environ["export MLFLOW_TRACKING_URI"]="https://dagshub.com/rajuu9825/End-to-end-Machine-Learning-Project-with-MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="rajuu9825"
os.environ["MLFLOW_TRACKING_PASSWORD"]="e9d3dc7e0e33d09d6992ea6a541706f25a7d1007"



In [6]:
# ENtity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
# prepare confuguration manager class
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
            config =  self.config.model_evaluation
            params = self.params.ElasticNet
            schema = self.schema.TARGET_COLUMN

            create_directories([config.root_dir])

            model_trainer_config = ModelEvaluationConfig(
                root_dir = config.root_dir,
                test_data_path =  config.test_data_path,
                model_path = config.model_path,
                all_params = params,
                metric_file_name=config.metric_file_name,
                target_column=schema.name,
                mlflow_uri="https://dagshub.com/rajuu9825/End-to-end-Machine-Learning-Project-with-MLFlow.mlflow"
            )

            return model_trainer_config

In [ ]:
!pip install mlflow        

^C


  Using cached mlflow-2.21.3-py3-none-any.whl.metadata (30 kB)
  Using cached mlflow_skinny-2.21.3-py3-none-any.whl.metadata (31 kB)
  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached matplotlib-3.10.1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached pyarrow-19.0.1-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
  Using cached sqlalchemy-2.0.40-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached waitress-3.0.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import os
import pandas as pd
from src.mlproject.utils.common import save_json
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlflow.models.signature import infer_signature

In [55]:
import dagshub
dagshub.init(repo_owner='rajuu9825', repo_name='End-to-end-Machine-Learning-Project-with-MLFlow', mlflow=True)


ModuleNotFoundError: No module named 'dagshub'

In [58]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        
        return rmse,mae,r2
    
    def log_into_mlflow(self):

        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Set MLflow Tracking URI (DagsHub or local)
        mlflow.set_tracking_uri(self.config.mlflow_uri)  # e.g., https://dagshub.com/your_username/your_repo.mlflow

        # Optional: set username/token as env vars if needed
        # os.environ["MLFLOW_TRACKING_USERNAME"] = "your_username"
        # os.environ["MLFLOW_TRACKING_PASSWORD"] = "your_token"

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Set or create experiment
        mlflow.set_experiment("wine-quality")

        with mlflow.start_run():

            # Predictions and signature
            predicted_qualities = model.predict(test_x)
            signature = infer_signature(test_x, predicted_qualities)

            # Evaluation
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            # Save metrics locally (optional)
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log params and metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Log model and register it
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(
                    sk_model=model,
                    artifact_path="model",
                    registered_model_name="WineQualityModel",  # This name will show in Model Registry
                    signature=signature
                )
            else:
                mlflow.sklearn.log_model(
                    sk_model=model,
                    artifact_path="model",
                    signature=signature
                )

            print("MLflow logging complete 🚀")


In [59]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-04-05 20:16:06,614: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-05 20:16:06,628: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-05 20:16:06,636: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-05 20:16:06,648: INFO: common: Created directory at: artifacts]
[2025-04-05 20:16:06,655: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-04-05 20:16:11,367: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'WineQualityModel' already exists. Creating a new version of this model...
2025/04/05 20:17:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: WineQualityModel, version 2
Created version '2' of model 'WineQualityModel'.


MLflow logging complete 🚀
🏃 View run useful-rook-911 at: https://dagshub.com/rajuu9825/End-to-end-Machine-Learning-Project-with-MLFlow.mlflow/#/experiments/2/runs/857c7c47772844ecb91b96755b910786
🧪 View experiment at: https://dagshub.com/rajuu9825/End-to-end-Machine-Learning-Project-with-MLFlow.mlflow/#/experiments/2


ModuleNotFoundError: No module named 'dagshub'